# Calculating the number of households within certain distances from tanks for each county using ```.sjoin()```

### Import statements

In [4]:
import warnings
warnings.filterwarnings('ignore')

In [9]:
import pandas as pd
import geopandas as gpd
import dask
from dask import dataframe as dd
import dask_geopandas

### Reading county shapefile dataframe
To count the number of households within five miles of a storage tank for each county, we use GeoPandas' ```.sjoin()``` method. Using this method, we will perform a spatial join between each county's geometry and the dataframe including Point geometries for each household in the US. For this, we need a dataframe with geometries for all counties in the US - which we took from the United States Census Bureau's Cartographic Boundary Files (available [here](https://www.census.gov/geographies/mapping-files/time-series/geo/carto-boundary-file.html)). Then, we filter to exclude counties from Alaska, Hawaii, Puerto Rico, Virgin Islands, American Samoa, Guam, Northern Marian Islands, as there are no tanks in those regions in the AST dataset. We also drop unnecessary columns.

In [10]:
df_counties = gpd.read_file('/hpc/group/codeplus22-vis/celine_data/source_files/county_shapefiles/counties.shp')
df_counties = df_counties[df_counties['STATEFP'] != '02']
df_counties = df_counties[df_counties['STATEFP'] != '15']
df_counties = df_counties[df_counties['STATEFP'] != '72']
df_counties = df_counties[df_counties['STATEFP'] != '78']
df_counties = df_counties[df_counties['STATEFP'] != '60']
df_counties = df_counties[df_counties['STATEFP'] != '66']
df_counties = df_counties[df_counties['STATEFP'] != '69']
df_counties = df_counties[['NAME', 'geometry']]
df_counties.rename(columns = {'NAME': 'county'}, inplace = True)
df_counties

county                                           geometry
0         Riley  POLYGON ((-96.96095 39.28670, -96.96106 39.288...
1      Ringgold  POLYGON ((-94.47167 40.81255, -94.47166 40.819...
2        Carbon  POLYGON ((-109.79867 45.16734, -109.68779 45.1...
3     Bear Lake  POLYGON ((-111.63452 42.57034, -111.63010 42.5...
4       Buffalo  POLYGON ((-92.08384 44.41200, -92.08310 44.414...
...         ...                                                ...
3229     Asotin  POLYGON ((-117.47999 46.12199, -117.41948 46.1...
3230    Candler  POLYGON ((-82.25457 32.35150, -82.25276 32.353...
3231  Tom Green  POLYGON ((-101.26763 31.55646, -101.25039 31.5...
3232    Licking  POLYGON ((-82.78181 39.94698, -82.78126 39.955...
3233   Woodbury  POLYGON ((-96.49878 42.56104, -96.46085 42.561...

[3108 rows x 2 columns]

### Reading InfoUSA Data (pre-processed)
This InfoUSA data was preprocessed to include the distance between each household and the nearest tank. We need this information to filter for only households within five miles of a storage tank, which we do below.

In [ ]:
df_hh = pd.read_parquet('/hpc/group/codeplus22-vis/celine_data/dist_all_hh_with_children_fixed.parquet')
df_hh.head()

We also only keep the ```lat_h_4326``` and ```lon_h_4326``` columns because these are the coordinates for each household we will use to create the Point geometries for each household in the households GeoDataFrame. We use the coordinates in EPSG 4326 rather than the coordinates in EPSG 3857 because the geometries in the ```df_counties``` GeoDataFrame are in EPSG 4326. The coordinate system must remain consistent in order to get accurate results when performing the spatial join with ```.sjoin()```.

In [ ]:
df_hh = df_hh[['lat_h_4326', 'lon_h_4326', 'distance_category']]
df_hh = df_hh[df_hh['distance_category'] != 4]
df_hh.head()

### Use Dask to transform pandas dataframe to a geopandas dataframe
To perform a spatial join between two dataframes, each of these dataframes must be GeoDataFrames- thus, we must convert our ```df_hh``` dataframe to a GeoDataFrame. However, as this dataframe has more than 17 million rows, converting it without using Dask is not time-efficient. Hence, we turned to Dask, an open-source Python library for parallel computing. It allows us to efficiently execute the transformation of our dataframe to a GeoDataFrame, even when working with over 17 million rows. 

To use Dask, we first converted our dataframe to a Dask dataframe, using Dask's ```.from_pandas()``` method. This method takes in our pandas dataframe along with the ```npartitions``` parameter, which is used to specify the number of 'sections' the dask dataframe will be split into.

In [ ]:
df_dask = dd.from_pandas(df_hh, npartitions = 500)

Then, we specify what manipulation to the dask dataframe ```df_dask``` to compute. In this case, we use Dask Geopandas' ```.points_from_xy()``` method to convert the pandas dask dataframe into a geopandas dask dataframe.

In [ ]:
%%time
df_dask['geometry'] = dask_geopandas.points_from_xy(df_dask, 'lon_h_4326', 'lat_h_4326')

After, we convert the dask geodataframe into a geopandas dataframe:

In [ ]:
%%time
gdf = dask_geopandas.from_dask_dataframe(df_dask)

Calling compute puts all the above code into action. Dask executes each set of commands on each partition, as specified above. This returns GeoDataFrame ```gdf_hh```, with over 17 million rows, in around five seconds.

In [ ]:
%%time
gdf_hh = gdf.compute()

In [ ]:
gdf_hh = gdf_hh[['distance_category', 'geometry']]
gdf_hh.head()

### Computing number of households within 0.5 mi of a storage tank in each county
Now that we have our two GeoDataFrames, we ```.sjoin()``` to find which households are within 0.5 miles from a tank in each county.

#### Filtering the household data for only households within 0.5 mi of a storage tank
First, we created a new GeoDataFrame ```gdf_hh_half_mi``` that included only the households within 0.5 miles of a storage tank.

In [15]:
gdf_hh_half_mi = gdf_hh[gdf_hh['distance_category'] == 1]
gdf_hh_half_mi.head()

distance_category                    geometry
3348                  1  POINT (-73.66289 40.99470)
3354                  1  POINT (-73.66757 41.00379)
3355                  1  POINT (-73.66548 40.99359)
3356                  1  POINT (-73.66699 41.00108)
3365                  1  POINT (-73.67412 41.00968)

#### Adding column in county dataframe with the number of households within 0.5mi of a tank
Then, we used ```.sjoin()``` to find the number of households within 0.5 miles of a tank for each county.

Calling ```.sjoin()``` and passing in two GeoDataFrames will return a new GeoDataFrame that only includes the observations with geometries that are the intersections of the two original GeoDataFrames. In our case, passing in a GeoDataFrame with the geometry for Harris County and a GeoDataFrame with all the households within 0.5 miles of a tank to the ```.sjoin()``` method returns a new GeoDataFrame with all the households within 0.5 miles of a tank in Harris County, as it returns all the Point geometries that intersects the Harris county Polygon geometry. Computing the length of that new dataframe through ```len()``` returns the number of households within 0.5 miles in Harris county, then.

However, we need to do this for all counties in the US, so we use for loop. This loop iterates every row of ```df_counties``` finds the intersection between that county and the households within 0.5 miles of a tank (```gdf_hh_half_mi```), computes the length of that ```intersect_df```, and adds that number to a new column in ```df_counties```. We add the column to ```df_counties``` specifically, because then we can use this exact dataframe to plot a map of the US where each county is colored by the number of households within five miles of a storage tank, as it contains all the geometries for each county required for a GeoViews visualization.

In [16]:
%%time

df_counties['hh_half_mi'] = 0
df_counties_temp = df_counties

for i in range(0, len(df_counties)):
    county = df_counties_temp.head(n=1)
    intersect_df = gpd.sjoin(county, gdf_hh_half_mi, how='inner', predicate='intersects')
    df_counties_temp = df_counties_temp.iloc[1:, :]
    num = len(intersect_df)
    # print(num)
    
    # adding num to df_counties column
    df_counties['hh_half_mi'].iloc[i] = num

df_counties

CPU times: user 13min 37s, sys: 1min 59s, total: 15min 37s
Wall time: 15min 42s


county                                           geometry  hh_half_mi
0       Ballard  POLYGON ((-89.18137 37.04630, -89.17938 37.053...           0
1       Bourbon  POLYGON ((-84.44266 38.28324, -84.44114 38.283...           0
2        Butler  POLYGON ((-86.94486 37.07341, -86.94346 37.074...           0
3        Estill  POLYGON ((-84.12662 37.64540, -84.12483 37.646...           0
4       Fleming  POLYGON ((-83.98428 38.44549, -83.98246 38.450...           0
...         ...                                                ...         ...
3228     Gosper  POLYGON ((-100.09510 40.43866, -100.08937 40.4...           0
3229     Holmes  POLYGON ((-82.22066 40.66758, -82.19327 40.667...           0
3230  Gillespie  POLYGON ((-99.30400 30.49983, -99.28234 30.499...           0
3231  Milwaukee  POLYGON ((-88.06959 42.86726, -88.06959 42.872...        1595
3232     Ottawa  POLYGON ((-86.26432 43.11830, -86.25103 43.118...        2725

[3108 rows x 3 columns]

In [7]:
df_counties = gpd.read_file('/hpc/group/codeplus22-vis/celine_data/hh_per_county.shp')
df_counties

county  hh_half_mi  hh_one_mi  hh_five_mi  hh_total  \
0         Riley           0          0           0         0   
1      Ringgold           0          0           0         0   
2        Carbon           0          0          89        89   
3     Bear Lake           0          0           0         0   
4       Buffalo           0          0           0         0   
...         ...         ...        ...         ...       ...   
3103     Asotin         366       1329        1236      2931   
3104    Candler           0          0           0         0   
3105  Tom Green         543       2295       11950     14788   
3106    Licking           9        707       14973     15689   
3107   Woodbury         573        739       13805     15117   

                                               geometry  
0     POLYGON ((-96.96095 39.28670, -96.96106 39.288...  
1     POLYGON ((-94.47167 40.81255, -94.47166 40.819...  
2     POLYGON ((-109.79867 45.16734, -109.68779 45.1...  
3     POLYGON ((-111.63452 42.57034, -111.63010 42.5...  
4     POLYGON ((-92.08384 44.41200, -92.08310 44.414...  
...                                                 ...  
3103  POLYGON ((-117.47999 46.12199, -117.41948 46.1...  
3104  POLYGON ((-82.25457 32.35150, -82.25276 32.353...  
3105  POLYGON ((-101.26763 31.55646, -101.25039 31.5...  
3106  POLYGON ((-82.78181 39.94698, -82.78126 39.955...  
3107  POLYGON ((-96.49878 42.56104, -96.46085 42.561...  

[3108 rows x 6 columns]

### Computing number of households within 1 mi of a storage tank in each county
Following the same steps as described above, except filtering for households within one mile of a storage tank.

#### Filtering the household data for only households within 1 mi of a storage tank

In [8]:
gdf_hh_one_mi = gdf_hh[gdf_hh['distance_category'] == 2]
gdf_hh_one_mi

NameError: name 'gdf_hh' is not defined

#### Adding column in county dataframe with the number of households within 1 mi of a tank
Takes around 30min

In [16]:
%%time

df_counties['hh_one_mi'] = 0
df_counties_temp = df_counties

for i in range(0, len(df_counties)):
    county = df_counties_temp.head(n=1)
    intersect_df = gpd.sjoin(county, gdf_hh_one_mi, how='inner', predicate='intersects')
    df_counties_temp = df_counties_temp.iloc[1:, :]
    num = len(intersect_df)
    # print(num)
    
    # adding num to df_counties column
    df_counties['hh_one_mi'].iloc[i] = num

df_counties


KeyboardInterrupt



In [20]:
df_counties.to_file('/hpc/group/codeplus22-vis/infousa_copy/hh_num_counties_all_final.shp')

In [3]:
df_counties = gpd.read_file('/hpc/group/codeplus22-vis/infousa_copy/hh_num_counties_all_final.shp')
df_counties

STATEFP COUNTYFP  COUNTYNS        AFFGEOID  GEOID       NAME LSAD  \
0         21      007  00516850  0500000US21007  21007    Ballard   06   
1         21      017  00516855  0500000US21017  21017    Bourbon   06   
2         21      031  00516862  0500000US21031  21031     Butler   06   
3         21      065  00516879  0500000US21065  21065     Estill   06   
4         21      069  00516881  0500000US21069  21069    Fleming   06   
...      ...      ...       ...             ...    ...        ...  ...   
3103      31      073  00835858  0500000US31073  31073     Gosper   06   
3104      39      075  01074050  0500000US39075  39075     Holmes   06   
3105      48      171  01383871  0500000US48171  48171  Gillespie   06   
3106      55      079  01581100  0500000US55079  55079  Milwaukee   06   
3107      26      139  01623012  0500000US26139  26139     Ottawa   06   

           ALAND      AWATER  HH_half_mi  HH_one_mi  HH_five_mi  HH_total  \
0      639387454    69473325           0          0           0         0   
1      750439351     4829777           0          0           0         0   
2     1103571974    13943044           0          0           0         0   
3      655509930     6516335           0          0           0         0   
4      902727151     7182793           0          0           0         0   
...          ...         ...         ...        ...         ...       ...   
3103  1186616237    11831826           0          0           0         0   
3104  1094405866     3695230           0          0           0         0   
3105  2740719114     9012764           0          0           0         0   
3106   625440563  2455383635        1595       8269      106533    116397   
3107  1459502408  2765830983        2725       5189       21621     29535   

                                               geometry  
0     POLYGON ((-89.18137 37.04630, -89.17938 37.053...  
1     POLYGON ((-84.44266 38.28324, -84.44114 38.283...  
2     POLYGON ((-86.94486 37.07341, -86.94346 37.074...  
3     POLYGON ((-84.12662 37.64540, -84.12483 37.646...  
4     POLYGON ((-83.98428 38.44549, -83.98246 38.450...  
...                                                 ...  
3103  POLYGON ((-100.09510 40.43866, -100.08937 40.4...  
3104  POLYGON ((-82.22066 40.66758, -82.19327 40.667...  
3105  POLYGON ((-99.30400 30.49983, -99.28234 30.499...  
3106  POLYGON ((-88.06959 42.86726, -88.06959 42.872...  
3107  POLYGON ((-86.26432 43.11830, -86.25103 43.118...  

[3108 rows x 14 columns]

### Computing number of households within 5mi of a storage tank in each county
Following the same steps as described above, except filtering for households within five miles of a storage tank. Because the households GeoDataFrame has over 14 million rows, this process takes quite a while.

#### Filtering the household data for only households within 5 mi of a storage tank

In [18]:
gdf_hh_five_mi = gdf_hh[gdf_hh['distance_category'] == 3]
gdf_hh_five_mi

distance_category                    geometry
2976                      3  POINT (-77.59820 43.08710)
2977                      3  POINT (-77.59820 43.08710)
2978                      3  POINT (-77.59820 43.08710)
2979                      3  POINT (-77.59820 43.08710)
2980                      3  POINT (-77.59820 43.08710)
...                     ...                         ...
53059098                  3  POINT (-98.58665 33.92580)
53059100                  3  POINT (-98.57742 33.91278)
53059101                  3  POINT (-98.33038 33.91823)
53059103                  3  POINT (-98.41964 33.89710)
53059107                  3  POINT (-98.32977 33.90988)

[14404277 rows x 2 columns]

#### Adding column in county dataframe with the number of households within 5km of a tank
Takes around 4 hours.

In [ ]:
%%time 

df_counties['hh_five_mi'] = 0
df_counties_temp = df_counties

for i in range(0, len(df_counties)):
    county = df_counties_temp.head(n=1)
    intersect_df = gpd.sjoin(county, gdf_hh_five_mi, how='inner', predicate='intersects')
    df_counties_temp = df_counties_temp.iloc[1:, :]
    num = len(intersect_df)
    
    # adding num to df_counties column 
    df_counties['hh_five_mi'].iloc[i] = num

df_counties

In [13]:
df_counties['HH_total'] = df_counties['HH_half_mi'] + df_counties['HH_one_mi'] + df_counties['HH_five_mi']
df_counties

STATEFP COUNTYFP  COUNTYNS        AFFGEOID  GEOID       NAME LSAD  \
0         21      007  00516850  0500000US21007  21007    Ballard   06   
1         21      017  00516855  0500000US21017  21017    Bourbon   06   
2         21      031  00516862  0500000US21031  21031     Butler   06   
3         21      065  00516879  0500000US21065  21065     Estill   06   
4         21      069  00516881  0500000US21069  21069    Fleming   06   
...      ...      ...       ...             ...    ...        ...  ...   
3103      31      073  00835858  0500000US31073  31073     Gosper   06   
3104      39      075  01074050  0500000US39075  39075     Holmes   06   
3105      48      171  01383871  0500000US48171  48171  Gillespie   06   
3106      55      079  01581100  0500000US55079  55079  Milwaukee   06   
3107      26      139  01623012  0500000US26139  26139     Ottawa   06   

           ALAND      AWATER  HH_half_mi  HH_one_mi  \
0      639387454    69473325           0          0   
1      750439351     4829777           0          0   
2     1103571974    13943044           0          0   
3      655509930     6516335           0          0   
4      902727151     7182793           0          0   
...          ...         ...         ...        ...   
3103  1186616237    11831826           0          0   
3104  1094405866     3695230           0          0   
3105  2740719114     9012764           0          0   
3106   625440563  2455383635        1595       8269   
3107  1459502408  2765830983        2725       5189   

                                               geometry  HH_five_mi  HH_total  
0     POLYGON ((-89.18137 37.04630, -89.17938 37.053...           0         0  
1     POLYGON ((-84.44266 38.28324, -84.44114 38.283...           0         0  
2     POLYGON ((-86.94486 37.07341, -86.94346 37.074...           0         0  
3     POLYGON ((-84.12662 37.64540, -84.12483 37.646...           0         0  
4     POLYGON ((-83.98428 38.44549, -83.98246 38.450...           0         0  
...                                                 ...         ...       ...  
3103  POLYGON ((-100.09510 40.43866, -100.08937 40.4...           0         0  
3104  POLYGON ((-82.22066 40.66758, -82.19327 40.667...           0         0  
3105  POLYGON ((-99.30400 30.49983, -99.28234 30.499...           0         0  
3106  POLYGON ((-88.06959 42.86726, -88.06959 42.872...      106533    116397  
3107  POLYGON ((-86.26432 43.11830, -86.25103 43.118...       21621     29535  

[3108 rows x 14 columns]

### Exporting it as a shapefile

In [ ]:
df_counties.to_file('/hpc/group/codeplus22-vis/infousa_copy/hh_num_counties_all.shp')

In [5]:
df = gpd.read_file('/hpc/group/codeplus22-vis/infousa_copy/hh_num_counties_all.shp')
df

STATEFP COUNTYFP  COUNTYNS        AFFGEOID  GEOID       NAME LSAD  \
0         21      007  00516850  0500000US21007  21007    Ballard   06   
1         21      017  00516855  0500000US21017  21017    Bourbon   06   
2         21      031  00516862  0500000US21031  21031     Butler   06   
3         21      065  00516879  0500000US21065  21065     Estill   06   
4         21      069  00516881  0500000US21069  21069    Fleming   06   
...      ...      ...       ...             ...    ...        ...  ...   
3103      31      073  00835858  0500000US31073  31073     Gosper   06   
3104      39      075  01074050  0500000US39075  39075     Holmes   06   
3105      48      171  01383871  0500000US48171  48171  Gillespie   06   
3106      55      079  01581100  0500000US55079  55079  Milwaukee   06   
3107      26      139  01623012  0500000US26139  26139     Ottawa   06   

           ALAND      AWATER  HH_half_mi  HH_one_mi  HH_five_mi  HH_total  \
0      639387454    69473325           0          0           0         0   
1      750439351     4829777           0          0           0         0   
2     1103571974    13943044           0          0           0         0   
3      655509930     6516335           0          0           0         0   
4      902727151     7182793           0          0           0         0   
...          ...         ...         ...        ...         ...       ...   
3103  1186616237    11831826           0          0           0         0   
3104  1094405866     3695230           0          0           0         0   
3105  2740719114     9012764           0          0           0         0   
3106   625440563  2455383635        1595       8269      106533    116397   
3107  1459502408  2765830983        2725       5189       21621     29535   

                                               geometry  
0     POLYGON ((-89.18137 37.04630, -89.17938 37.053...  
1     POLYGON ((-84.44266 38.28324, -84.44114 38.283...  
2     POLYGON ((-86.94486 37.07341, -86.94346 37.074...  
3     POLYGON ((-84.12662 37.64540, -84.12483 37.646...  
4     POLYGON ((-83.98428 38.44549, -83.98246 38.450...  
...                                                 ...  
3103  POLYGON ((-100.09510 40.43866, -100.08937 40.4...  
3104  POLYGON ((-82.22066 40.66758, -82.19327 40.667...  
3105  POLYGON ((-99.30400 30.49983, -99.28234 30.499...  
3106  POLYGON ((-88.06959 42.86726, -88.06959 42.872...  
3107  POLYGON ((-86.26432 43.11830, -86.25103 43.118...  

[3108 rows x 14 columns]